### Part 1 - Student details:
* Please write the First Name and last 4 digits of the i.d. for each student. For example:
<pre>Israel 7938</pre>

In [ ]:
# student 1: Israel 7938
# student 2: Eran 6215
# student 3: Avihai 2474
# student 4: Almog 8269 TESTTTTTTTTTTTTTTTTTT
# student 5: Liron 5068
# student 6: Matan ####

SyntaxError: invalid syntax (1739801622.py, line 1)

## Part 2 - Experiments
You could add as many code cells as needed

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sklearn

In [13]:
url = "https://raw.githubusercontent.com/eranCat/TitanicML/main/titanic_test.csv"
testset = pd.read_csv(url)

In [14]:
trainset.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survived
0,3,1,35.000000,0,0,7.0500,0,0
1,3,1,30.000000,0,0,7.2500,0,0
2,3,1,29.699118,0,0,14.5000,0,0
3,3,0,21.000000,0,0,7.7500,2,0
4,3,1,32.000000,0,0,56.4958,0,1


In [15]:
url = "https://raw.githubusercontent.com/eranCat/TitanicML/main/titanic_train.csv"
trainset = pd.read_csv(url)

In [16]:
trainset.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survived
0,1,0,54.0,1,0,78.2667,1,1
1,2,0,45.0,0,0,13.5000,0,1
2,3,1,20.0,0,0,9.5000,0,0
3,3,1,36.0,0,0,7.8958,0,0
4,3,1,22.0,0,0,8.0500,0,0


## Part 3 - Training flow
Use the best combination of feature engineering, model (algorithm and hyperparameters) from the experiment part (part 2)

## Part 4 - Apply on test and show model performance estimation